# Modelo Matemático - Empresa de Aviação

Importação dos pacotes JuMP e Gurobi, além da instanciação do modelo.

In [ ]:
#!/usr/bin/env  julia
using JuMP
using Gurobi

m = Model(with_optimizer(Gurobi.Optimizer))

### Dados

Declaração das constantes que representam os dados do problema.

- $tipos\_avioes = \{1, 2, 3\}$;
- $tipos\_avioes_{1}$: Boeing 717;
- $tipos\_avioes_{2}$: Boeing 737-500;
- $tipos\_avioes_{3}$: MD-11;



In [ ]:
# dados
tipos_avioes = collect(1:3)
tamanho_avioes = [1, 3/4, 5/3]

verba = 220
max_galpao = 40

custos = [5.1, 3.6, 6.8]
receitas = [330, 300, 420]
pilotos = [30, 20, 10]

### Variáveis

Declaração das variáveis do problema: quantidade a ser comprada de cada avião, sendo que todos têm de ser valores maior igual a zero e inteiros.

$quantidades = (quantidade_{1}, quantidade_{2}, quantidade_{3}) | \forall i \in tipos\_avioes, quantidades_{i} \ge 0$

In [ ]:
# variaveis
@variable(m, quantidades[tipos_avioes] >= 0, Int)

### Definição da função objetivo

O objetivo da função é maximizar o lucro considerando a receita e o custo de compra de cada avião. Assim sendo, temos a seguinte função objetivo:

\begin{equation}
    \max f(q) = \sum_{i \in q} q[i] * (r[i] - c[i]), \\
    q = (quantidades[i]), i \in tipos\_avioes \\
    r = (receitas[i]), i \in tipos\_avioes \\
    c = (custos[i]), i \in tipos\_avioes \\
\end{equation}

In [ ]:
# funcao objetivo
@objective(m, Max, sum(quantidades[i] * (receitas[i] - custos[i]) for i in tipos_avioes))

### Restrições

Considerando que
\begin{equation}
    z = (tipos\_avioes[i]), i \in tipos\_avioes \\
    t = (tamanho\_avioes[i]), i \in tipos\_avioes \\
    q = (quantidades[i]), i \in tipos\_avioes \\
    c = (custos[i]), i \in tipos\_avioes \\
    p = (pilotos[i]), i \in tipos\_avioes \\
    g = max\_galpao \\
    v = verba \\
\end{equation}
As restrições para o problema são as que seguem abaixo:

-  As oficinas de manutenção podem suportar até 40 Boeings 717. Um Boeing 737-500 equivale, em esforço de manutenção, a 3/4, e um MD-11 a 5/3, quando referidos ao Boeing 717. Temos então que:
  \begin{equation}
      \sum_{i \in z} t[i] * q[i] <= g;
  \end{equation}

-  O custo total não deve superar a verba de compra:
  \begin{equation}
      \sum_{i \in z} c[i] * q[i] <= v;
  \end{equation}

-  Os pilotos de MD-11 podem pilotar todos os aviões da empresa, mas os demais pilotos só podem ser escalados às aeronaves a que foram habilitados. Cada aeronave necessita de dois pilotos para operar e não se deve comprar mais aviões do que a quantidade que pode ser usada ao mesmo tempo. Assim sendo, (i) a quantidade de aviões MD-11 deve ser menor que metade do número de pilotos; (ii) a quantidade de aviões Boeing 717 somada à quantidade de aviões MD-11 deve ser menor que a metade de pilotos do primeiro tipo somada a quantidade de pilotos do segundo; (iii) a quantidade de aviões Boeing 737-500 somada à quantidade de aviões MD-11 deve ser menor que a metade de pilotos do primeiro somada a quantidade de pilotos do segundo.
  \begin{equation}
      \qquad 2 * q[3] <= p[3]; \qquad (i) \\
      \qquad 2 * q[1] + 2 * q[3] <= p[1] + p[3]; \qquad (ii) \\
      \qquad 2 * q[2] + 2 * q[3] <= p[2] + p[3]; \qquad (iii) \\
  \end{equation}

In [ ]:
# resticoes
@constraints(m,
    begin
        sum(tamanho_avioes[i] * quantidades[i] for i in tipos_avioes) <= max_galpao
        sum(custos[i] * quantidades[i] for i in tipos_avioes) <= verba
        2 * quantidades[3] <= pilotos[3]
        2 * quantidades[1] + 2 * quantidades[3] <= pilotos[1] + pilotos[3]
        2 * quantidades[2] + 2 * quantidades[3] <= pilotos[2] + pilotos[3]
    end
)

### Modelo

Com todas as definições feitas pode-se obter o modelo completo prara este problema, que pode ser visto a partir da execução do código abaixo.

In [ ]:
# imprime modelo
println(m)

### Execução

Tendo-se o modelo apresentado, pode-se obter o resultado ótimo para as quantidades com a execução do código abaixo.

In [ ]:
# obtém o resultado ótimo
start_time = time()
optimize!(m)
end_time = time()

# escreve os resultados
println("Tempo: $(end_time - start_time) s")
println("Resultado função: ", JuMP.objective_value(m))

custo_total = 0
for i in tipos_avioes
    if (JuMP.value(quantidades[i]) > 0)
        println("quantidades[", i, "]: ", JuMP.value(quantidades[i]))
        println("Custo: ", JuMP.value(quantidades[i]) * custos[i])
        global custo_total += JuMP.value(quantidades[i]) * custos[i]
    elseif (JuMP.value(quantidades[i]) == 0)
        println("quantidades[", i, "]: ", 0.0)
    end
end
println("Custo total: ", custo_total)

println()